In [1]:
# general immports may go here!
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import List, Optional, Tuple, Dict
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [2]:
def initialize_graph(G, attr="state"):
    """
    :param G: networkx graph
    :param attr: string specifying the node attribute 
    :
    :return: resulting networkx graph
    """
    # reducing the network to its biggest component
    H = max(nx.connected_component_subgraphs(G), key=len)
    intialization={node: 0 for node in G.nodes()} #susceptible
    nx.set_node_attributes(H,intialization,attr)
    patient_zero=random.choice(list(H.nodes())) #randomly select a node
    H.nodes[patient_zero][attr]=1 #change its state to 1 - infected
    return H

In [3]:
def spread_SIS(G: nx.Graph, beta: float, mu: float, attr: str="state") -> nx.Graph:
    """
    :param G: input networkx graph
    :param beta: probability that an infected individual infects one of his neighbors
    :param mu: probability that an infected individual has recovered at the next timestep
    :param attr: string specifying the name of the node attribute which indicates whether 
    :            or not the corresponding individual is infected
    :             -> should take value 0 if the corresponding individual susceptible, 
    :                and 1 if the individual is infected
    :return: the resulting network after the disease has spread for one round
    """
    
    node_states=nx.get_node_attributes(G,attr)
    
    attr2 = 'recover'
    def infected(G,attr,node_states,beta):   
        for v in G.nodes():
            if node_states[v]==0:
                for w in G[v]:
                    if G.nodes[w][attr]==1:
                        if random.random() < beta:
                            node_states[v]=1
                            print('the patient infected:', v)
                            break
        nx.set_node_attributes(G,node_states,attr)

    def get_recover_patient(G, mu, node_states, attr2):
        node_recover = nx.get_node_attributes(G,attr2)
        for i in G.nodes():
            if node_states[i]==1:
                a = random.random()
                print('p:', a)
                if a < mu:
                    node_recover[i]=1
                else:
                    node_recover[i]=0
        nx.set_node_attributes(G,node_states,attr2)
#         recover_list =[x for x,y in G.nodes(data=True) if y['recover'] == 1]
#         print('recover list:', recover_list)

   
    def change_infected_into_recover(G,node_states,attr,attr2):
        node_recover = nx.get_node_attributes(G,attr2)
        for x in G.nodes():
            if node_recover[x] == 1:
                node_states[x]=0
#         print('actually recover:', recover_list)
        recover_list =[x for x,y in G.nodes(data=True) if y['recover'] == 1]
        print('recover list:', recover_list)
        nx.set_node_attributes(G,node_states,attr)
        return G
        
    infected(G, attr, node_states, beta)
    get_recover_patient(G, mu, node_states, attr2)
    change_infected_into_recover(G,node_states,attr, attr2)
    
#     recover_num = len([x for x,y in G.nodes(data=True) if y['recover'] == 0])
#     if recover_num == 100:
#         infected(G, attr, node_states, beta)
#         list_recover = get_recover_patient(G, mu, node_states, attr2)
#     else:
        
#         change_infected_into_recover(G,node_states,list_recover,attr)
#         infected(G, attr, node_states, beta)
#         list_recover = get_recover_patient(G, mu, node_states, attr2)
    
    
    return G

In [4]:
BA = nx.barabasi_albert_graph(10,2)
BA0 = initialize_graph(BA, 'state')
BA0_new = nx.set_node_attributes(BA0, name = 'recover',values =0)
label = nx.get_node_attributes(BA0,'recover')
print(BA0.node(data=True))

[(0, {'state': 0, 'recover': 0}), (1, {'state': 0, 'recover': 0}), (2, {'state': 0, 'recover': 0}), (3, {'state': 0, 'recover': 0}), (4, {'state': 0, 'recover': 0}), (5, {'state': 0, 'recover': 0}), (6, {'state': 0, 'recover': 0}), (7, {'state': 0, 'recover': 0}), (8, {'state': 1, 'recover': 0}), (9, {'state': 0, 'recover': 0})]


In [5]:
H = BA0.copy()


In [6]:
spread_SIS(H,0.5, 0.2, 'state')
print(H.node(data=True))

the patient infected: 3
the patient infected: 6
p: 0.11186860375061836
p: 0.7557762508271718
p: 0.3589986294620966
recover list: [3, 6, 8]
[(0, {'state': 0, 'recover': 0}), (1, {'state': 0, 'recover': 0}), (2, {'state': 0, 'recover': 0}), (3, {'state': 0, 'recover': 1}), (4, {'state': 0, 'recover': 0}), (5, {'state': 0, 'recover': 0}), (6, {'state': 0, 'recover': 1}), (7, {'state': 0, 'recover': 0}), (8, {'state': 0, 'recover': 1}), (9, {'state': 0, 'recover': 0})]


In [7]:
# infected(H,'state', 0.5)
# print(H.node(data=True))

In [8]:
# get_recover_patient(H,0.8,'state','recover')
# print(H.node(data=True))